# This is a collection of python examples for 
# Numerical Optimization (Prof. Dr. Volker Schulz)

# chapter 2: SVM example code

In [ ]:
#%matplotlib
# from scikit learn documentation
#
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.datasets import make_blobs, make_classification


# we create 50 separable points in blobs
X, y = make_blobs(n_samples=50, centers=2, random_state=3)

# fit the model, don't regularize for illustration purposes
clf = svm.SVC(kernel='linear', C=1)
# C >= 1 for hard margin
# C = 0.01 for soft margin

# create not nicely separable points
#X, y = make_classification(n_samples=50,n_features=2, n_redundant=0, n_informative=2,
#                            n_clusters_per_class=1,random_state=10)
#clf = svm.SVC(kernel='rbf', C=1)
# choose from ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, 

clf.fit(X, y)

# after being fitted, the model can then be used to predict new values:
print(clf.predict([[-6, -2]]))
print(clf.predict([[-2, -1]]))
print(clf.predict([[2, 1]])) 
print(clf.predict([[5, 6]])) 


plt.scatter(X[:, 0], X[:, 1], c=y, s=30, cmap=plt.cm.Paired)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 50)
yy = np.linspace(ylim[0], ylim[1], 50)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = clf.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(XX, YY, Z, colors='k', levels=[-1, 0, 1], alpha=0.5,
           linestyles=['--', '-', '--'])
# plot support vectors
ax.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1], s=100,
           linewidth=1, facecolors='none', edgecolors='k')
plt.show()


## chapter 2: 3D pic for SVM illustration

In [ ]:
#%matplotlib
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

s2=np.sqrt(2.0)

L, n = 2.5, 400
x = np.linspace(-L, L, n)
y = x.copy()
X, Y = np.meshgrid(x, y)


# Compute random points
N = 100
rho = np.random.rand(N)
phi = 2 * np.pi * np.random.rand(N)
xb = rho * np.cos(phi)
yb = rho * np.sin(phi)
rho = np.random.rand(N)+1.5
phi = 2 * np.pi * np.random.rand(N)
xr = rho * np.cos(phi)
yr = rho * np.sin(phi)


# Set up a figure twice as wide as it is tall
fig = plt.figure(figsize=plt.figaspect(0.5))
ax = fig.add_subplot(1, 2, 1)
ax.plot(xb, yb, 'bo',
        xr, yr, 'ro')
ax.grid(True)
ax = fig.add_subplot(1, 2, 2, projection='3d')
surf = ax.plot_surface(X*X, Y*Y, s2*X*Y, rstride=20, cstride=20, cmap=cm.hot, alpha=0.25)
ax.scatter(xb*xb, yb*yb, s2*xb*yb, color = 'blue')
ax.scatter(xr*xr, yr*yr, s2*xr*yr, color = 'red')

plt.show()

## chapter 3:  illustration of convergence speeds

In [ ]:
from numpy import *
import math
import matplotlib.pyplot as plt

Max = 200
t = range(1, Max)
a = 0.9**(power(2,t))        # quadratic       red
b=ones(size(t))
b[0]=0.9
for k in range(1, Max-1):
    b[k] = b[k-1]/sqrt(k)    # superlinear     blue
c = power(0.9,t)             # linear          green
d = 0.9/(power(t,2))         # sublinear 2     cyan 
e = [0.9/k for k in t]       # sublinear 1     magenta
f = 0.9/sqrt(t)              # sublinear 1/2   black

#plt.xscale("log")
plt.yscale("log")
plt.ylim(10**(-12), 1)
plt.plot(t, a, 'r') 
plt.plot(t, b, 'b')  
plt.plot(t, c, 'g')  
plt.plot(t, d, 'c')  
plt.plot(t, e, 'm')  
plt.plot(t, f, 'k') 
plt.show()

## chapter 3: [from steepest descent to CG](From_SD_to_CG.ipynb)

## chapter 3: simple CG implementation

In [ ]:
import numpy as np
from scipy.sparse.linalg import cg
import time


def conjugate_grad(Q, b, x=None):
    """
    Description
    ----------
    Solve a linear-quadratic problem min 1/2 x^TQx-b^Tx with conjugate gradient method with Q spd.
    Parameters
    ----------
    Q: 2d numpy.array of positive semi-definite (symmetric) matrix
    b: 1d numpy.array
    x: 1d numpy.array of initial point
    Returns
    -------
    1d numpy.array x such that Qx = b
    """
    n = len(b)
    if not x:
        x = np.ones(n)
    r = np.dot(Q, x) - b
    p = - r
    r_k_norm = np.dot(r, r)
    for i in range(2*n):
        Qp = np.dot(Q, p) # can be replaced by a function evaluation of Q times p
        alpha = r_k_norm / np.dot(p, Qp)
        x += alpha * p
        r += alpha * Qp
        r_kplus1_norm = np.dot(r, r)
        beta = r_kplus1_norm / r_k_norm
        r_k_norm = r_kplus1_norm
        if r_kplus1_norm < 1e-8:
            print ('Itr:', i)
            break
        p = beta * p - r
    return x


def prec_conjugate_grad(Q, W, b, x=None):
    """
    Description
    ----------
    Solve a linear-quadratic problem min 1/2 x^TQx-b^Tx with preconditioned conjugate gradient method with Q spd.
    Parameters
    ----------
    Q: 2d numpy.array of positive semi-definite (symmetric) matrix
    W: 2d numpy.array preconditioning matrix
    b: 1d numpy.array
    x: 1d numpy.array of initial point
    Returns
    -------
    1d numpy.array x such that Qx = b
    """
    n = len(b)
    if not x:
        x = np.ones(n)
    r = np.dot(Q, x) - b
    z = np.dot(W, r)
    p = - z
    r_k_z = np.dot(r, z)
    for i in range(2*n):
        Qp = np.dot(Q, p)
        alpha = r_k_z / np.dot(p, Qp)
        x += alpha * p
        r += alpha * Qp
        z = np.dot(W, r)
        r_k_z_plus1 = np.dot(r, z)
        beta = r_k_z_plus1 / r_k_z
        r_k_z = r_k_z_plus1
        if np.dot(r, r) < 1e-8:
            print ('Itr:', i)
            break
        p = beta * p - z
    return x

if __name__ == '__main__':
    n =1000
    P = np.random.normal(size=[n, n])
    Q = np.dot(P.T, P)
    Q += 10000.0*np.diag(np.diag(Q))
    b = np.ones(n)

    t1 = time.time()
    print ('start')
    x = conjugate_grad(Q, b)
    t2 = time.time()
    print (t2 - t1)
    x2 = np.linalg.solve(Q, b)
    t3 = time.time()
    print (t3 - t2)
    x3 = cg(Q, b)
    t4 = time.time()
    print (t4 - t3)
    #W=np.identity(n)
    W=np.linalg.inv(np.diag(np.diag(Q))) # Jacobi preconditioner
    #W=np.linalg.inv(Q)
    t4 = time.time()
    x = prec_conjugate_grad(Q, W, b)
    t5 = time.time()
    print (t5 - t4)
    print (np.linalg.norm(np.dot(Q, x) - b))



## chapter 3: use CG to solve 1D heat equation

Sparse and symmetric matrix:

$A=\left[
\begin{array}{c}
-2F & F\\
F & -2F & F \\
  & \ddots & \ddots & \ddots\\
   &        &  F     & -2F  
   \end{array}
\right]
$

In [ ]:
import numpy as np
import scipy.sparse.linalg as ssl
import matplotlib.pyplot as plt


def callback(xk):
    global num_iters
    num_iters += 1

Nx=3000
F=(Nx+1)*(Nx+1);A = np.zeros((Nx, Nx))
for i in range(1, Nx-1):
    A[i,i-1]=-F;A[i,i]=2*F;A[i,i+1]=-F
A[0,0]= 2*F; A[0,1]=-F
A[Nx-1,Nx-1]= 2*F; A[Nx-1,Nx-2]= -F

b=np.ones(Nx)
x0=np.zeros(Nx)

I=np.identity(Nx)
A_upper_diag = np.triu(A, k=0)

Aupinv= np.linalg.inv(A_upper_diag)
M=0.5*(Aupinv+Aupinv.T)

print(A)
#print(np.linalg.eigvalsh(A))

print(A_upper_diag)

#with efficient incomplete LU (ilu) preconditioning
num_iters = 0
A_x = lambda x: A@x         # python lambda function used
Aop = ssl.LinearOperator((Nx, Nx), A_x)
Milu = ssl.spilu(A,options=dict(SymmetricMode=True))
M_x = lambda x: Milu.solve(x) # python lambda function used
Mop = ssl.LinearOperator((Nx, Nx), M_x)
x, info = ssl.cg(Aop, b, x0=x0, M=Mop, maxiter=2000, tol=1e-5,callback=callback)
print("# cg iter's with efficient ILU preconditioning")
print(num_iters)

#with simple preconditioning
num_iters = 0
x, info = ssl.cg(A, b, x0=x0, M=M, maxiter=2000, tol=1e-5,callback=callback)
print("# cg iter's with simple preconditioning")
print(num_iters)

#without preconditioning
num_iters = 0
x, info = ssl.cg(A, b, x0=x0, M=I, maxiter=2000, tol=1e-5,callback=callback)
print("# cg iter's without preconditioning")
print(num_iters)

## Chapter 3: Newton vs steepest descent (vs Quasi-Newton [comp_QN = 1])

In [ ]:
import numpy as np
import numpy.linalg as la
import scipy.optimize as sopt

import matplotlib.pyplot as pt
from mpl_toolkits.mplot3d import axes3d

# choose problem type from
# quadratic, rosenbrock
problem = "rosenbrock"

#choose step from
# steepest, Newton 
step = "Newton"

#choose comparison with quasi Newton
# steepest, Newton 
comp_QN = 1 # 1: switches on



def f(x):
    if problem == "quadratic":
        return 0.5*x[0]**2 + 2.5*x[1]**2
    elif problem == "rosenbrock":
        return (1.0-x[0])**2 + 100.0*(x[1]-x[0]**2)**2
    else:
        return 0

def df(x):
    if problem == "quadratic":
        return np.array([x[0], 5*x[1]])
    elif problem == "rosenbrock":
        return np.array([-2.0*(1.0-x[0]) - 400.0*(x[1]-x[0]**2)*x[0], 200.0*(x[1]-x[0]**2)])
    else:
        return 0


def f1d(alpha):
    return f(x + alpha*s)

def quad_hess(x):
     H = np.array([[1.0,0.0],[0.0,5.0]])
     return H

def rosen_hess(x):
     x = np.asarray(x)
     H = np.diag(-400*x[:-1],1) - np.diag(400*x[:-1],-1)
     diagonal = np.zeros_like(x)
     diagonal[0] = 1200*x[0]**2-400*x[1]+2
     diagonal[-1] = 200
     diagonal[1:-1] = 202 + 1200*x[1:-1]**2 - 400*x[2:]
     H = H + np.diag(diagonal)
     return H

def ddf(x):
    if problem == "quadratic":
        return quad_hess(x)
    elif problem == "rosenbrock":
        return rosen_hess(x)
    else:
        return 0
    
def callback(xk):
    guesses.append(xk)
    return 0

# Set up a figure twice as wide as it is tall
fig = pt.figure(figsize=pt.figaspect(0.5))

# Set up two subplots: left for 3d view, right for contour
ax = fig.add_subplot(1, 2, 1, projection='3d')

xmesh, ymesh = np.mgrid[-2:2:75j,-2:2:75j]
fmesh = f(np.array([xmesh, ymesh]))
ax.plot_surface(xmesh, ymesh, fmesh)

ax = fig.add_subplot(1, 2, 2)

pt.axis("equal")
pt.contour(xmesh, ymesh, fmesh)

guesses = [np.array([2, 2])] # 2,3./5

x = guesses[-1]

if step == "steepest":
    maxit = 2000
elif step == "Newton":
    maxit = 15
else:
    print("choose optimization method")



for i in range(maxit):
    s = -df(x)
    if step == "Newton":
        H=ddf(x)
        s=la.solve(H,s)
    if np.linalg.norm(s) < 0.0001:
        break # step length as stopping criterion
    alpha_opt = sopt.golden(f1d) #line search

    x = x + alpha_opt * s
    next_guess = x
    guesses.append(next_guess)
#    print(next_guess)

print("number of optimization steps:", i)
pt.axis("equal")
pt.contour(xmesh, ymesh, fmesh, 150)
it_array = np.array(guesses)
pt.plot(it_array.T[0], it_array.T[1], "x-")

#compare with L-BFGS Quasi-Newton
if comp_QN == 1:
    guesses = [np.array([2, 2])] # 2,3./5
    x = guesses[-1]
    sopt.minimize(f, x, method="L-BFGS-B", jac=df, tol=0.00001, callback=callback, options={'maxcor': 2, 'maxiter': 100})
    # c.f. https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html
    it_array = np.array(guesses)
    pt.plot(it_array.T[0], it_array.T[1], "x-")
    print("number of QN steps:", it_array.shape[0])

pt.show()


## Chapter 3: Rosenbrock as nonlinear least squares

In [ ]:
import numpy as np
from scipy.optimize import least_squares

def fun_rosenbrock(x):
    return np.array([10 * (x[1] - x[0]**2), (1 - x[0])])

def jac_rosenbrock(x):
    return np.array([
        [-20 * x[0], 10],
        [-1, 0]])

x0_rosenbrock = np.array([2, 2])

res_2 = least_squares(fun_rosenbrock, x0_rosenbrock, jac_rosenbrock)
# c.f. https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html#scipy.optimize.least_squares
print('solution: ',res_2.x)
print('final objective: ',res_2.cost)
print('final norm of grad: ',res_2.optimality)
print('# fcn eval.: ',res_2.nfev)
print('# jac eval.: ',res_2.njev)
                  

## Chapter 3: Alternating least squares for explicit collaborative filtering

In [ ]:
# data from http://files.grouplens.org/datasets/movielens/
# program from https://github.com/danielnee/Notebooks/blob/master/ALS/ALS_Explicit.ipynb
# see also http://danielnee.com/2016/09/collaborative-filtering-using-alternating-least-squares/


import pandas as pd
import numpy as np

SimpleExample = True # False

ratingsNames = ["userId", "movieId", "rating", "timestamp"]
usersNames = ["userId", "gender", "age", "occupation", "zipCode"]
moviesNames = ["movieId", "title", "genres"]

if SimpleExample:
    ratings = pd.read_table("simple-exp/rating1.dat", header=None, sep="::", names=ratingsNames)
    users = pd.read_table("simple-exp/user1.dat", header=None, sep="::", names=usersNames)
    movies = pd.read_table("simple-exp/mov1.dat", header=None, sep="::", names=moviesNames)
    f=3; iters = 100
    regLamba = 0.01
else:
    ratings = pd.read_table("ml-1m/ratings.dat", header=None, sep="::", names=ratingsNames)
    users = pd.read_table("ml-1m/users.dat", header=None, sep="::", names=usersNames)
    movies = pd.read_table("ml-1m/movies.dat", header=None, sep="::", names=moviesNames)
    f=20; iters = 10
    regLamba = 0.1

print(movies.head())

n = max(movies.movieId)
m = max(users.userId)

def normaliseRow(x):
    return x / sum(x)

def initialiseMatrix(n, f):
    A = abs(np.random.randn(n, f))
    return np.apply_along_axis(normaliseRow, 1, A)

# Initialise Y matrix, n x f
Y = initialiseMatrix(n, f)
# Initialise X matrix, m x f
X = initialiseMatrix(m, f)

# Create a dummy entry for each movie
temp = np.zeros((n, 4))
for i in range(1, n):
    temp[i,] = [m+1, i, 0, 0]
    
ratings = ratings.append(pd.DataFrame(temp, columns =ratingsNames))

ratingsMatrix = ratings.pivot_table(columns=['movieId'], index =['userId'], values='rating', dropna = False)

ratingsMatrix = ratingsMatrix.fillna(0).values  #ratingsMatrix.fillna(0).as_matrix() deprecated

# Drop the dummy movie
ratingsMatrix = ratingsMatrix[1:m+1,1:n+1]

def ratingsPred(X, Y):
    return np.dot(X, Y.T)

def MSE(ratingsPred, ratingsMatrix):
    idx = ratingsMatrix > 0
    return sum((ratingsPred[idx] - ratingsMatrix[idx]) ** 2) / np.count_nonzero(ratingsMatrix)
    
print(MSE(ratingsPred(X, Y), ratingsMatrix))

nonZero = ratingsMatrix > 0

reg = regLamba * np.eye(f,f)

for k in range(1, iters):
    for i in range(1, m):
        idx = nonZero[i,:]
        a = Y[idx,]
        b = np.dot(np.transpose(Y[idx,]), ratingsMatrix[i, idx])
        updateX = np.linalg.solve(np.dot(np.transpose(a), a) + reg, b)
        X[i,] = updateX
    
    for j in range(1, n):
        idx = nonZero[:,j]
        a = X[idx,]
        b = np.dot(np.transpose(X[idx,]), ratingsMatrix[idx, j])
        updateY = np.linalg.solve(np.dot(np.transpose(a), a) + reg, b)
        Y[j,] = updateY
        
    ratingsP = ratingsPred(X, Y)
    mse = MSE(ratingsP, ratingsMatrix)
    print("MSE: " + str(mse))
    
if SimpleExample:
    print(ratingsP)        
    print(ratingsMatrix)        
print("Done")


## Chapter 4: Examples of artificial neural networks

In [ ]:
# Variation of 
# tensor flow for beginners from https://www.tensorflow.org/tutorials/quickstart/beginner 
Fashion_MNIST = True # False

from matplotlib import pyplot
import tensorflow as tf

# your python installation has to be enhanced by tensorflow 2 using one of the two methods:
# a) pip install tensorflow      -> if you want to use your standard CPU, mostly done here
# b) pip install tensorflow-gpu  -> if you want to use your GPU, should be nVidia with CUDA compatibility >= 3.5
print("TensorFlow version: ",tf.__version__)

# Load and prepare the MNIST dataset. Convert the samples from integers to floating-point numbers
mnist = tf.keras.datasets.mnist
fashion_mnist = tf.keras.datasets.fashion_mnist

if Fashion_MNIST:
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
else:
    (x_train, y_train), (x_test, y_test) = mnist.load_data()



if Fashion_MNIST:
    x_test_store = x_test

# summarize loaded dataset
print('\n Train size: X=%s, y=%s' % (x_train.shape, y_train.shape))
print(' Test  size: X=%s, y=%s' % (x_test.shape, y_test.shape))
# plot first few train images
print("\n The first 9 training pictures; ")
for i in range(9):
    # define subplot
    pyplot.subplot(330 + 1 + i)
    # plot raw pixel data
    pyplot.imshow(x_train[i], cmap=pyplot.get_cmap('gray'))
# show the figure
pyplot.show()

if not Fashion_MNIST:
# plot first few test images
    print("\n The first 9 test pictures: ")
    for i in range(9):
        # define subplot
        pyplot.subplot(330 + 1 + i)
        # plot raw pixel data
        pyplot.imshow(x_test[i], cmap=pyplot.get_cmap('gray'))
    # show the figure
    pyplot.show()

x_train, x_test = x_train / 255.0, x_test / 255.0

input_shape = (28, 28, 1)


#Build the tf.keras.Sequential model by stacking layers. 
# find more layers on https://keras.io/api/layers/
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10) # defaul activation None, i.e. linear
])


model.summary()



#For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.
predictions = model(x_train[:1]).numpy()
print("\n Prediction vector example for first training picture")
print(predictions)

#The tf.nn.softmax function converts these logits to "probabilities" for each class:
print("\n Probability vector example for first training picture")
print(tf.nn.softmax(predictions).numpy())

#Choose a loss function for training
#The losses.SparseCategoricalCrossentropy loss takes a vector of logits and a True index and returns a scalar loss for each example.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

#This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.
#This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.log(1/10) ~= 2.3.
print("\n Loss example for first training picture")
print(loss_fn(y_train[:1], predictions).numpy())

#Choose an optimizer 
#for default values, see https://keras.io/api/optimizers/
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

#The Model.fit method adjusts the model parameters to minimize the loss:
model.fit(x_train, y_train, epochs=10)

#The Model.evaluate method checks the models performance, usually on a "Validation-set" or "Test-set".
model.evaluate(x_test,  y_test, verbose=2)

#let us look at probabilities, wrap the trained model, and attach the softmax to it:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])


if Fashion_MNIST:
    eval_labels = tf.keras.backend.argmax(probability_model(x_test[:30]))
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    pyplot.figure(figsize=(10,10))
    for i in range(25):
        pyplot.subplot(5,5,i+1)
        pyplot.xticks([])
        pyplot.yticks([])
        pyplot.grid(False)
        pyplot.imshow(x_test_store[i], cmap=pyplot.cm.binary)
        pyplot.xlabel(class_names[eval_labels[i]])
    pyplot.show()
    
else:
    #print("\n Probability vectors for first 9 test images")
    #print(probability_model(x_test[:9]))
    print("\n Argmax vector for first 9 test images")
    print(tf.keras.backend.argmax(probability_model(x_test[:9])))


## Chapter 4: challenging neural network based on CNN for fashion MNIST

In [ ]:
#tensorflow_version 2.x
# Hifi CNN with multiple layers
# see for other NN architectures: https://github.com/zalandoresearch/fashion-mnist
# use GPU, e.g. on https://colab.research.google.com/notebooks/gpu.ipynb
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, Dropout, BatchNormalization, MaxPooling2D, Flatten, Dense


def make_model():
  model = Sequential()
  model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', input_shape=(28,28,1)))
  model.add(BatchNormalization())

  model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))

  model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))
    
  model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=1, padding='same'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(128, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  model.add(Dense(10, activation='softmax'))
  return model

model = make_model()

model.summary()

fashion_mnist = tf.keras.datasets.fashion_mnist
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

print (train_images.shape)
print (train_labels.shape)
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

optimizer = tf.keras.optimizers.Adam (lr=0.001)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

#model.fit(train_images, train_labels, epochs=30, callbacks=[reduce_lr])
model.fit(train_images, train_labels,
          batch_size=100,
          epochs=30,
          callbacks=[reduce_lr],
          verbose=1)

test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

## Chapter 5. Some QP Solvers

In [ ]:
from numpy import array, dot
from IPython.display import display, Markdown, Latex
from qpsolvers import solve_qp, available_solvers
#sudo -H pip install qpsolvers  # comes with quadprog from Goldfarb
#sudo -H pip install cvxopt  # interior point method
#sudo -H pip install osqp  #admm
#https://pypi.org/project/qpsolvers/

#choose QP solver from: CVXOPT, qpOASES, quadprog, ECOS as wrapped by CVXPY, Gurobi, MOSEK, OSQP
#QP in standard form according to https://scaron.info/teaching/quadratic-programming.html
#

M = array([[1., 2., 0.], [-8., 3., 2.], [0., 1., 1.]])
P = dot(M.T, M)  # quick way to build a symmetric matrix
q = dot(array([3., 2., 3.]), M).reshape((3,))
G = array([[1., 2., 1.], [2., 0., 1.], [-1., 2., -1.]])
h = array([3., 2., -2.]).reshape((3,))
A = array([1., 1., 1.])
b = array([1.])

print("QP:")
display(Latex('$\min {1\over 2} x^T Px+q^T x$'))
display(Latex('s.t. $Ax = b$'))
display(Latex('and $Gx\leq h$'))

print("where:")

print("\n P = "), print(P)
print("\n q = "), print(q)
print("\n A = "), print(A)
print("\n b = "), print(b)
print("\n G = "), print(G)
print("\n h = "), print(h)


print("\n\n")


print("options for solvers: ", available_solvers)

print("QP solution:")
print("x = ",solve_qp(P, q, G, h, A, b,solver="cvxopt"))



## Chapter 5: Markowitz portfolio optimization

This is the portfolio Jupyter notebook from  https://www.quantopian.com/posts/a-tutorial-on-markowitz-portfolio-optimization-in-python-using-cvxopt
    but updated to Python 3. There, you may find also a section on backtesting on real market data, which I omit here.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd

np.random.seed(123)

# Turn off progress printing 
solvers.options['show_progress'] = False

Assume that we have 4 assets, each with a return series of length 1000. We can use numpy.random.randn to sample returns from a normal distribution.

In [ ]:
## NUMBER OF ASSETS
n_assets = 4

## NUMBER OF OBSERVATIONS
n_obs = 1000

return_vec = np.random.randn(n_assets, n_obs)

plt.plot(return_vec.T, alpha=.4);
plt.xlabel('time')
plt.ylabel('returns')

These return series can be used to create a wide range of portfolios, which all have different returns and risks (standard deviation). We can produce a wide range of random weight vectors and plot those portfolios. As we want all our capital to be invested, this vector will have to some to one.

In [ ]:
def rand_weights(n):
    ''' Produces n random weights that sum to 1 '''
    k = np.random.rand(n)
    return k / sum(k)

print(rand_weights(n_assets))
print(rand_weights(n_assets))

Next, lets evaluate how many of these random portfolios would perform. Towards this goal we are calculating the mean returns as well as the volatility (here we are using standard deviation). You can also see that there is a filter that only allows to plot portfolios with a standard deviation of < 2 for better illustration.

In [ ]:
def random_portfolio(returns):
    ''' 
    Returns the mean and standard deviation of returns for a random portfolio
    '''

    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # This recursion reduces outliers to keep plots pretty
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma

In the code you will notice the calculation of the return with:
$$ R = p^T w $$
where $R$ is the expected return, $p^T$ is the transpose of the vector for the mean returns for each time series and w is the weight vector of the portfolio. $p$ is a Nx1 column vector, so $p^T$ turns into a 1xN row vector which can be multiplied with the Nx1 weight (column) vector w to give a scalar result. This is equivalent to the dot product used in the code. Keep in mind that Python has a reversed definition of rows and columns and the accurate NumPy version of the previous equation would be R = w * p.T
Next, we calculate the standard deviation with
$$\sigma = \sqrt{w^T C w}$$
where $C$ is the covariance matrix of the returns which is a NxN matrix. Please note that if we simply calculated the simple standard deviation with the appropriate weighting using std(array(ret_vec).T*w) we would get a slightly different ’bullet’. This is because the simple standard deviation calculation would not take covariances into account. In the covariance matrix, the values of the diagonal represent the simple variances of each asset while the off-diagonals are the variances between the assets. By using ordinary std() we effectively only regard the diagonal and miss the rest. A small but significant difference.
Lets generate the mean returns and volatility for 500 random portfolios:

In [ ]:
n_portfolios = 500
means, stds = np.column_stack([
    random_portfolio(return_vec) 
    for _ in range(n_portfolios)
])

Upon plotting those you will observe that they form a characteristic parabolic shape called the ‘Markowitz bullet‘ with the boundaries being called the ‘efficient frontier‘, where we have the lowest variance for a given expected.

In [ ]:
plt.plot(stds, means, 'o', markersize=5)
plt.xlabel('std')
plt.ylabel('mean')
plt.title('Mean and standard deviation of returns of randomly generated portfolios')

Markowitz optimization and the Efficient Frontier

Once we have a good representation of our portfolios as the blue dots show we can calculate the efficient frontier Markowitz-style. This is done by minimising
$$ w^T C w$$
for $w$ on the expected portfolio return $R^T w$ whilst keeping the sum of all the weights equal to 1:
$$ \sum_{i}{w_i} = 1 $$
Here we parametrically run through $R^T w = \mu$ and find the minimum variance for different $\mu$‘s. This can be done with scipy.optimise.minimize but we have to define quite a complex problem with bounds, constraints and a Lagrange multiplier. Conveniently, the cvxopt package, a convex solver, does all of that for us. We used one of their examples with some modifications as shown below. You will notice that there are some conditioning expressions in the code. They are simply needed to set up the problem. For more information please have a look at the cvxopt example.
The mus vector produces a series of expected return values $\mu$ in a non-linear and more appropriate way. We will see later that we don‘t need to calculate a lot of these as they perfectly fit a parabola, which can safely be extrapolated for higher values.

In [ ]:
def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns)
    
    N = 100
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis=1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))   # negative n x n identity matrix
    h = opt.matrix(0.0, (n ,1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x'] 
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    # CALCULATE THE OPTIMAL PORTFOLIO -> CALL the QP solver CVXOPT
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(wt), returns, risks

weights, returns, risks = optimal_portfolio(return_vec)

plt.plot(stds, means, 'o')
plt.ylabel('mean')
plt.xlabel('std')
plt.plot(risks, returns, 'y-o')

In yellow you can see the optimal portfolios for each of the desired returns (i.e. the mus). In addition, we get the one optimal portfolio returned:

In [ ]:
print(weights)

## Chapter 6: Lasso with ADMM

In [ ]:
import pdb,time
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from numpy.linalg import norm,cholesky

"""
https://github.com/afbujan/admm_lasso
Author  : Alex Bujan (adapted from http://www.stanford.edu/~boyd)
Date    : 12/06/2015
06/30/2020 : ported to Python 3 and slighty changed (Volker Schulz)
"""


def lasso_admm(X,y,alpha,rho=1.,rel_par=1.,QUIET=False,\
                MAX_ITER=50,ABSTOL=1e-3,RELTOL= 1e-2):
    """
     Solve lasso problem via ADMM
    
     [z, history] = lasso_admm(X,y,alpha,rho,rel_par)
    
     Solves the following problem via ADMM:
    
       minimize 1/2*|| Ax - y ||_2^2 + alpha || x ||_1
    
     The solution is returned in the vector z.
    
     history is a dictionary containing the objective value, the primal and
     dual residual norms, and the tolerances for the primal and dual residual
     norms at each iteration.
    
     rho is the augmented Lagrangian parameter.
    
     rel_par is the over-relaxation parameter (typical values for rel_par are
     between 1.0 and 1.8).
    
     More information can be found in the paper linked at:
     http://www.stanford.edu/~boyd/papers/distr_opt_stat_learning_admm.html
    """

    if not QUIET:
        tic = time.time()

    #Data preprocessing
    m,n = X.shape
    #save a matrix-vector multiply
    Xty = X.T.dot(y)

    #ADMM solver
    x = np.zeros((n,1))
    z = np.zeros((n,1))
    u = np.zeros((n,1))

    # cache the (Cholesky) factorization
    L,U = factor(X,rho)

    if not QUIET:
        print ('\n%3s\t%10s\t%10s\t%10s\t%10s\t%10s' % ('iter',
                                                      'r norm', 
                                                      'eps pri', 
                                                      's norm', 
                                                      'eps dual', 
                                                      'objective'))

    # Saving state
    h = {}
    h['objval']     = np.zeros(MAX_ITER)
    h['r_norm']     = np.zeros(MAX_ITER)
    h['s_norm']     = np.zeros(MAX_ITER)
    h['eps_pri']    = np.zeros(MAX_ITER)
    h['eps_dual']   = np.zeros(MAX_ITER)

    for k in range(MAX_ITER):

        # x-update 
        q = Xty+rho*(z-u) #(temporary value)
        if m>=n:
            x = spsolve(U,spsolve(L,q))[...,np.newaxis]
        else:
            ULXq = spsolve(U,spsolve(L,X.dot(q)))[...,np.newaxis]
            x = (q*1./rho)-((X.T.dot(ULXq))*1./(rho**2))

        # z-update with relaxation
        zold = np.copy(z)
        x_hat = rel_par*x+(1.-rel_par)*zold
        z = shrinkage(x_hat+u,alpha*1./rho)

        # u-update
        u+=(x_hat-z)

        # diagnostics, reporting, termination checks
        h['objval'][k]   = objective(X,y,alpha,x,z)
        h['r_norm'][k]   = norm(x-z)
        h['s_norm'][k]   = norm(-rho*(z-zold))
        h['eps_pri'][k]  = np.sqrt(n)*ABSTOL+\
                            RELTOL*np.maximum(norm(x),norm(-z))
        h['eps_dual'][k] = np.sqrt(n)*ABSTOL+\
                            RELTOL*norm(rho*u)
        if not QUIET:
            print ('%4d\t%10.4f\t%10.4f\t%10.4f\t%10.4f\t%10.2f' % (k+1,\
                                                          h['r_norm'][k],\
                                                          h['eps_pri'][k],\
                                                          h['s_norm'][k],\
                                                          h['eps_dual'][k],\
                                                          h['objval'][k]))

        if (h['r_norm'][k]<h['eps_pri'][k]) and (h['s_norm'][k]<h['eps_dual'][k]) and (k>30):
            break

    if not QUIET:
        toc = time.time()-tic
        print ("\nElapsed time is %.2f seconds" % toc)

    return z.ravel(),h

def objective(X,y,alpha,x,z):
    return .5*np.square(X.dot(x)-y).sum()+alpha*norm(z,1)

def shrinkage(x,kappa):
    return np.maximum(0.,x-kappa)-np.maximum(0.,-x-kappa)

def factor(X,rho):
    m,n = X.shape
    if m>=n:
       L = cholesky(X.T.dot(X)+rho*sparse.eye(n))
    else:
       L = cholesky(sparse.eye(m)+1./rho*(X.dot(X.T)))
    L = sparse.csc_matrix(L)
    U = sparse.csc_matrix(L.T)
    return L,U

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sparse
from numpy.linalg import norm


"""
Author  : Alex Bujan (adapted from http://www.stanford.edu/~boyd)
Date    : 12/06/2015
06/30/2020 : ported to Python 3 and slighty changed (Volker Schulz)
"""

np.random.seed(1234)

m = 1500        # number of examples
n = 5000        # number of features
p = 100/n       # sparsity density

x0  = sparse.rand(n,1,p)
X   = np.random.randn(m,n)
D   = sparse.diags(1./np.sqrt(np.sum(X**2,0)).T,0,(n,n))
X   = D.T.dot(X.T).T
y   = x0.T.dot(X.T).T+np.sqrt(1e-3)*np.random.randn(m,1)


alpha_max = norm(X.T.dot(y),np.inf)
alpha = .1*alpha_max

print ('\n*Alpha: %.4f' % alpha)

x, h = lasso_admm(X,y,alpha,1.,1.)

K = len(h['objval'][np.where(h['objval']!=0)])

fig1 = plt.figure(1)
ax = fig1.add_subplot(111)
ax.plot(np.arange(K), h['objval'][:K],'k',ms=10,lw=2)
ax.set_ylabel('f(x^k) + g(z^k)')
ax.set_xlabel('iter (k)')

fig2 = plt.figure(2)
ax1 = fig2.add_subplot(211)
ax1.semilogy(np.arange(K),np.maximum(1e-8,h['r_norm'][:K]),'k',lw=2)
ax1.semilogy(np.arange(K),h['eps_pri'][:K],'k--',lw=2)
ax1.set_ylabel('||r||_2')

ax2 = fig2.add_subplot(212)
ax2.semilogy(np.arange(K),np.maximum(1e-8,h['s_norm'][:K]),'k',lw=2)
ax2.semilogy(np.arange(K),h['eps_dual'][:K],'k--',lw=2)
ax2.set_ylabel('||s||_2')
ax2.set_xlabel('iter (k)')

plt.show()

## Chapter 6: SQP for a simple problem
see documentation in https://docs.scipy.org/doc/scipy-0.19.0/reference/tutorial/optimize.html#tutorial-sqlsp

In [ ]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# objective function
def func(x, sign=1.0):
    return sign*(2*x[0]*x[1] + 2*x[0] - x[0]**2 - 2*x[1]**2)

#derivative of objective function
def func_deriv(x, sign=1.0):
    dfdx0 = sign*(-2*x[0] + 2*x[1] + 2)
    dfdx1 = sign*(2*x[0] - 4*x[1])
    return np.array([ dfdx0, dfdx1 ])
    
cons = ({'type': 'eq',
          'fun' : lambda x: np.array([x[0]**3 - x[1]]),
          'jac' : lambda x: np.array([3.0*(x[0]**2.0), -1.0])},
         {'type': 'ineq', # >= 0
          'fun' : lambda x: np.array([x[1] - 1]),
          'jac' : lambda x: np.array([0.0, 1.0])})

# unconstrained minimization
res = minimize(func, [-1.0,1.0], args=(-1.0,), jac=func_deriv,
                method='SLSQP', options={'disp': True})
print(res.x)

#constrained minimization
res = minimize(func, [-1.0,1.0], args=(-1.0,), #jac=func_deriv,
                constraints=cons, method='SLSQP', options={'disp': True})
print(res.x)

## Chapter 6: SQP nonnegative low rank approximation of matrices

In [ ]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

np.set_printoptions(precision=2)

print('\033[1m'+"Explicit recommendation matrix [<=0 means no observation]"+'\033[0m')

# define customer matrix from chapter 3.6
A_goal = np.array([[4,1,1,4], [1,4,2,0],[2,1,4,5],[1,4,1,-0.1]])
print(A_goal)
m = A_goal.shape[0]
n = A_goal.shape[1]

k_rank = 2  # rank of the low rank approximation

numvar = m*k_rank + k_rank*n

x = np.random.rand(numvar,1)
x = np.reshape(x,numvar) 

xrand = x

# objective function
def func(x):
    regfac = 0.1 # regularitation factor, sometimes called lambda
    x_split = np.split(x,[m*k_rank, numvar])
    U = np.reshape(x_split[0],(n,k_rank))
    V = np.reshape(x_split[1],(m,k_rank))
    A = U@V.T
    obj = 0.0
    for i in range(m):
        for j in range(n):
            if A_goal[i,j] >= 0.0 :
                obj += (A_goal[i,j]-A[i,j])**2
    obj += regfac*np.linalg.norm(U, 'fro')**2
    obj += regfac*np.linalg.norm(V, 'fro')**2
    return obj

# unconstrained minimization
print('\033[1m'+"\n"+"Unconstrained optimization result:"+'\033[0m')

res = minimize(func, x, method='SLSQP', options={'disp': True})

x_split = np.split(res.x,[m*k_rank, numvar])
recmat = np.reshape(x_split[0],(n,k_rank))@np.reshape(x_split[1],(m,k_rank)).T
print(recmat)

# constrained minimization

print('\033[1m'+"\n"+"Nonnegatively constrained optimization result:"+'\033[0m')

def confunc(x):
    x_split = np.split(x,[m*k_rank, numvar])
    A = np.reshape(x_split[0],(n,k_rank))@np.reshape(x_split[1],(m,k_rank)).T
    return np.reshape(A,m*n)

cons = {'type': 'ineq', # >=0
        'fun' : lambda x: confunc(x)}

res = minimize(func, xrand, constraints=cons, method='SLSQP', options={'disp': True})

x_split = np.split(res.x,[m*k_rank, numvar])
recmat = np.reshape(x_split[0],(n,k_rank))@np.reshape(x_split[1],(m,k_rank)).T
print(recmat)




